<a href="https://colab.research.google.com/github/paulyu8868/test/blob/main/%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8v5(%EC%98%A4%EB%A5%98%ED%95%B4%EA%B2%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings

# UserWarning 무시
warnings.filterwarnings("ignore", category=UserWarning)


def calculate_indicators(df):
    """향상된 기술적 지표 계산 - 인덱스 접근 수정"""
    df = df.copy()

    # ===== 1. 추세/모멘텀 지표 =====

    # 다양한 기간 이동평균과 이격도
    for period in [20, 50, 200]:
        # 이동평균 계산
        ma_values = df['Close'].rolling(window=period).mean()
        df[f'MA{period}'] = ma_values

        # 이격도 계산 (열 이름 확인 및 수정)
        if period in [50, 200]:  # 필요한 경우에만 생성
            temp_deviation = ((df['Close'] - ma_values) / ma_values * 100)
            df[f'MA{period}이격도'] = temp_deviation

    # MACD 계산
    temp_exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    temp_exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    temp_macd = temp_exp1 - temp_exp2
    df['MACD'] = temp_macd
    temp_signal = temp_macd.ewm(span=9, adjust=False).mean()
    df['Signal'] = temp_signal
    df['MACD_Hist'] = temp_macd - temp_signal

    # 다양한 기간 수익률
    for period in [5, 10, 20, 60]:
        temp_return = df['Close'].pct_change(period) * 100
        df[f'수익률_{period}'] = temp_return

    # ===== 2. 변동성 지표 =====

    # ATR
    temp_high_low = df['High'] - df['Low']
    temp_high_close = np.abs(df['High'] - df['Close'].shift())
    temp_low_close = np.abs(df['Low'] - df['Close'].shift())
    temp_ranges = pd.concat([temp_high_low, temp_high_close, temp_low_close], axis=1)
    temp_true_range = temp_ranges.max(axis=1)
    df['ATR'] = temp_true_range.rolling(window=14).mean()

    # Bollinger Bands 계산
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    df['BB_Std'] = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
    df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']).replace([np.inf, -np.inf], np.nan)

    # 변동성
    df['변동성_20'] = (df['Close'].rolling(window=20).std() / df['Close'] * 100)

    # ===== 3. 거래량 지표 =====

    # 거래량 증감률
    df['거래량_증감'] = df['Volume'].pct_change() * 100

    # 거래량 이동평균 대비
    df['거래량_MA20'] = df['Volume'].rolling(window=20).mean()
    temp_vol_ratio = (df['Volume'] / df['Volume'].rolling(window=20).mean())
    df['거래량_비율'] = temp_vol_ratio.replace([np.inf, -np.inf], np.nan)

    # OBV
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()

    # ===== 4. 가격 패턴 =====

    # 고가/저가 대비 현재가 위치
    for period in [20, 60]:
        temp_high = df['High'].rolling(window=period).max()
        temp_low = df['Low'].rolling(window=period).min()
        temp_position = ((df['Close'] - temp_low) / (temp_high - temp_low) * 100)
        df[f'가격위치_{period}'] = temp_position.replace([np.inf, -np.inf], np.nan)

    # 캔들 크기
    temp_candle_size = (df['High'] - df['Low']) / df['Close'] * 100
    df['캔들크기'] = temp_candle_size.replace([np.inf, -np.inf], np.nan)

    # 연속 상승/하락일수
    df['일간상승'] = np.where(df['Close'] > df['Close'].shift(1), 1, -1)
    temp_streak = df['일간상승'].groupby((df['일간상승'] != df['일간상승'].shift(1)).cumsum()).cumcount() + 1
    df['연속상승'] = temp_streak * df['일간상승']

    # RSI
    temp_delta = df['Close'].diff()
    temp_gain = (temp_delta.where(temp_delta > 0, 0)).rolling(window=14).mean()
    temp_loss = (-temp_delta.where(temp_delta < 0, 0)).rolling(window=14).mean()
    temp_rs = temp_gain / temp_loss
    df['RSI'] = 100 - (100 / (1 + temp_rs))

    # NaN 값 처리
    df = df.ffill().bfill()
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.ffill().bfill()

    return df





In [50]:
def get_data(ticker, start, end):
    """주가 데이터 불러오기 - 날짜 처리 최종 수정"""
    # 문자열 날짜를 datetime으로 변환
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)

    # yfinance에서 데이터 가져오기
    df = yf.download(ticker, start=start_dt, end=end_dt)

    # 거래량이 0인 행 제거
    df = df[df['Volume'] > 0].copy()

    # 인덱스의 timezone 제거 및 문자열로 변환
    df.index = pd.to_datetime(df.index).tz_localize(None).strftime('%Y-%m-%d')

    # 호가 단위 0.01$ 적용
    price_cols = ['Open', 'High', 'Low', 'Close']
    for col in price_cols:
        df[col] = df[col].round(2)

    # 등락율 계산
    df['Return'] = df['Close'].pct_change() * 100
    df['Return'] = df['Return'].round(2)

    # NaN 값 처리
    df = df.ffill().bfill()

    return df

def create_training_features(df, df_trades, target_return=1.1):
    """향상된 학습 데이터 생성"""
    features_list = []

    for _, trade in df_trades.iterrows():
        # 매수일을 문자열로 변환하여 데이터프레임 인덱스와 비교
        buy_date = pd.to_datetime(trade['매수일']).strftime('%Y-%m-%d')

        # 날짜 존재 여부 확인
        if buy_date not in df.index:
            print(f"Error: {buy_date} not in data index")
            continue

        try:
            trade_data = df.loc[buy_date]

            features = {
                '회차': trade['회차'],
                'MACD': float(trade_data['MACD']),
                'MACD_Hist': float(trade_data['MACD_Hist']),
                'MA50이격도': float(trade_data['MA50이격도']),
                'MA200이격도': float(trade_data['MA200이격도']),
                '수익률_5': float(trade_data['수익률_5']),
                '수익률_60': float(trade_data['수익률_60']),
                'ATR': float(trade_data['ATR']),
                'BB_Width': float(trade_data['BB_Width']),
                '변동성_20': float(trade_data['변동성_20']),
                '거래량_증감': float(trade_data['거래량_증감']),
                '거래량_비율': float(trade_data['거래량_비율']),
                '가격위치_20': float(trade_data['가격위치_20']),
                '가격위치_60': float(trade_data['가격위치_60']),
                '캔들크기': float(trade_data['캔들크기']),
                '연속상승': float(trade_data['연속상승']),
                'RSI': float(trade_data['RSI']),
                '실제수익률': trade['수익률(%)'],
                'is_target_achieved': trade['수익률(%)'] >= (target_return - 1) * 100
            }

            features_list.append(features)

        except Exception as e:
            print(f"Error processing trade {trade['회차']}: {str(e)}")
            continue

    print("\n[디버깅] 생성된 features_list 길이:", len(features_list))
    return pd.DataFrame(features_list)




def analyze_feature_importance(model, feature_names):
    """특성 중요도 분석"""
    importance = pd.DataFrame({
        '특성': feature_names,
        '중요도(%)': model.feature_importances_ * 100
    }).sort_values('중요도(%)', ascending=False)

    return importance

def analyze_model_performance(model, X_validate_scaled, y_validate, probability_threshold=0.5):
    """모델 성능 분석"""
    # 예측 확률
    y_prob = model.predict_proba(X_validate_scaled)[:, 1]

    # threshold 기반 예측
    y_pred = (y_prob >= probability_threshold).astype(int)

    # 기본 메트릭
    metrics = {
        '정확도': accuracy_score(y_validate, y_pred),
        '정밀도': precision_score(y_validate, y_pred),
        '재현율': recall_score(y_validate, y_pred),
        'F1': f1_score(y_validate, y_pred)
    }

    # 확률 구간별 실제 성공률
    prob_analysis = []
    prob_bins = np.arange(0, 1.1, 0.1)

    for i in range(len(prob_bins)-1):
        mask = (y_prob >= prob_bins[i]) & (y_prob < prob_bins[i+1])
        if mask.sum() > 0:
            actual_rate = y_validate[mask].mean()
            count = mask.sum()
            prob_analysis.append({
                '확률구간': f"{prob_bins[i]:.1f}~{prob_bins[i+1]:.1f}",
                '거래수': count,
                '실제성공률': actual_rate
            })

    return metrics, pd.DataFrame(prob_analysis)

def infinite_buy_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period):
    """매매 시뮬레이션 - 날짜 처리 최종 수정"""
    # 기존 코드와 동일하게 유지하되 날짜 처리만 문자열로 통일
    funds = initial_funds
    one_buy_amount = initial_funds / buy_portion
    holdings = 0
    buy_records = []
    trade_history = []
    trade_id = 1

    df_res = pd.DataFrame(index=range(start_idx, start_idx + simulation_period + 1),
                         columns=['날짜', '시가', '고가', '종가', '등락율',
                                'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                '보유 주식 수', '예수금', '총 평가액', '수익율(%)'])

    for i in range(start_idx, start_idx + simulation_period + 1):
        current_date = df.index[i]  # 이미 문자열 형식
        open_price = float(df['Open'].iloc[i])
        high_price = float(df['High'].iloc[i])
        close_price = float(df['Close'].iloc[i])
        return_val = float(df['Return'].iloc[i])
        prev_price = float(df['Close'].iloc[i-1]) if i > 0 else close_price

        df_res.at[i, '날짜'] = current_date  # 문자열 그대로 사용
        df_res.at[i, '시가'] = open_price
        df_res.at[i, '고가'] = high_price
        df_res.at[i, '종가'] = close_price
        df_res.at[i, '등락율'] = f"{return_val}%"

        price = close_price

        df_res.at[i, 'LOC 매수'] = 0
        df_res.at[i, '수익 실현 매도'] = 0
        df_res.at[i, 'MOC 손절'] = 0

        # 매수 로직
        if price <= prev_price*1.5:
            qty = int(one_buy_amount / price)
            if funds >= qty * price:
                holdings += qty
                funds -= qty * price
                buy_records.append({
                    'id': trade_id,
                    'buy_date': current_date,
                    'buy_price': price,
                    'quantity': qty,
                    'days': 0,
                    'type': 'LOC 매수'
                })
                df_res.at[i, 'LOC 매수'] = qty
                trade_id += 1

        # 매도 로직
        if holdings > 0:
            new_buy_records = []
            total_sell = 0

            for record in buy_records:
                record['days'] += 1
                sell_price = record['buy_price'] * 1.1

                if price >= sell_price:
                    funds += record['quantity'] * price
                    total_sell += record['quantity']
                    holdings -= record['quantity']
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round((price/record['buy_price'] - 1) * 100, 2)
                    })
                elif record['days'] >= 30:
                    funds += record['quantity'] * price
                    df_res.at[i, 'MOC 손절'] = record['quantity']
                    holdings -= record['quantity']
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round((price/record['buy_price'] - 1) * 100, 2)
                    })
                else:
                    new_buy_records.append(record)

            if total_sell > 0:
                df_res.at[i, '수익 실현 매도'] = total_sell

            buy_records = new_buy_records

        df_res.at[i, '보유 주식 수'] = holdings
        df_res.at[i, '예수금'] = round(funds, 2)
        df_res.at[i, '총 평가액'] = round(funds + (price * holdings), 2)
        df_res.at[i, '수익율(%)'] = round(((funds + (price * holdings)) / initial_funds - 1) * 100, 2)

    final_value = funds + (holdings * float(df['Close'].iloc[start_idx + simulation_period]))
    return round((final_value / initial_funds - 1) * 100, 2), df_res, final_value, pd.DataFrame(trade_history)


def ml_infinite_buy_simulation(df, df_res, initial_funds, buy_portion, start_idx, simulation_period,
                             model, scaler, prediction_threshold):
    """ML 기반 매매 시뮬레이션"""
    funds = initial_funds
    one_buy_amount = initial_funds / buy_portion
    holdings = 0
    buy_records = []
    trade_history = []
    trade_id = 1

    df_res = pd.DataFrame(index=range(start_idx, start_idx + simulation_period + 1),
                         columns=['날짜', '시가', '고가', '종가', '등락율',
                                'LOC 매수', '수익 실현 매도', 'MOC 손절',
                                '보유 주식 수', '예수금', '총 평가액', '수익율(%)',
                                'ML예측확률'])

    for i in range(start_idx, start_idx + simulation_period + 1):
        current_date = df.index[i]
        open_price = float(df['Open'].iloc[i])
        high_price = float(df['High'].iloc[i])
        close_price = float(df['Close'].iloc[i])
        return_val = float(df['Return'].iloc[i])
        prev_price = float(df['Close'].iloc[i-1]) if i > 0 else close_price

        df_res.at[i, '날짜'] = current_date
        df_res.at[i, '시가'] = open_price
        df_res.at[i, '고가'] = high_price
        df_res.at[i, '종가'] = close_price
        df_res.at[i, '등락율'] = f"{return_val}%"

        price = close_price

        df_res.at[i, 'LOC 매수'] = 0
        df_res.at[i, '수익 실현 매도'] = 0
        df_res.at[i, 'MOC 손절'] = 0

        # 매수 로직
        if price <= prev_price*1.5:
            # 현재 상태의 특성 계산
            current_data = df.iloc[i]
            # ml_infinite_buy_simulation 함수에서
            # '이격도_50' -> 'MA50이격도'로 수정
            features = [
                float(current_data['MACD']),
                float(current_data['MACD_Hist']),
                float(current_data['MA50이격도']),  # 수정된 부분
                float(current_data['MA200이격도']),
                float(current_data['수익률_5']),
                float(current_data['수익률_60']),
                float(current_data['ATR']),
                float(current_data['BB_Width']),
                float(current_data['변동성_20']),
                float(current_data['거래량_증감']),
                float(current_data['거래량_비율']),
                float(current_data['가격위치_20']),
                float(current_data['가격위치_60']),
                float(current_data['캔들크기']),
                float(current_data['연속상승']),
                float(current_data['RSI'])
            ]

            # 성공 확률 예측
            features_scaled = scaler.transform([features])
            success_prob = model.predict_proba(features_scaled)[0][1]
            df_res.at[i, 'ML예측확률'] = f"{success_prob:.1%}"

            # 예측 확률이 threshold를 넘을 때만 매수
            # 임의 수정 부분
            if success_prob >= prediction_threshold:
                qty = int(one_buy_amount / price)
                print(f'예측확률 : {success_prob}'f' >= {prediction_threshold}')
                if funds >= qty * price:
                    holdings += qty
                    funds -= qty * price
                    buy_records.append({
                        'id': trade_id,
                        'buy_date': current_date,
                        'buy_price': price,
                        'quantity': qty,
                        'days': 0,
                        'type': 'LOC 매수',
                        'pred_prob': success_prob
                    })
                    df_res.at[i, 'LOC 매수'] = qty
                    trade_id += 1

        # 매도 로직 (기존과   )
        if holdings > 0:
            new_buy_records = []
            total_sell = 0

            for record in buy_records:
                record['days'] += 1
                sell_price = record['buy_price'] * 1.1

                if price >= sell_price:
                    funds += record['quantity'] * price
                    total_sell += record['quantity']
                    holdings -= record['quantity']
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round((price/record['buy_price'] - 1) * 100, 2),
                        '예측확률': record['pred_prob']
                    })
                elif record['days'] >= 30:
                    funds += record['quantity'] * price
                    df_res.at[i, 'MOC 손절'] = record['quantity']
                    holdings -= record['quantity']
                    trade_history.append({
                        '회차': record['id'],
                        '매수일': record['buy_date'],
                        '매수가': record['buy_price'],
                        '매수수량': record['quantity'],
                        '매도일': current_date,
                        '매도가': price,
                        '매도수량': record['quantity'],
                        '보유기간': record['days'],
                        '수익률(%)': round((price/record['buy_price'] - 1) * 100, 2),
                        '예측확률': record['pred_prob']
                    })
                else:
                    new_buy_records.append(record)

            if total_sell > 0:
                df_res.at[i, '수익 실현 매도'] = total_sell

            buy_records = new_buy_records

        df_res.at[i, '보유 주식 수'] = holdings
        df_res.at[i, '예수금'] = round(funds, 2)
        df_res.at[i, '총 평가액'] = round(funds + (price * holdings), 2)
        df_res.at[i, '수익율(%)'] = round(((funds + (price * holdings)) / initial_funds - 1) * 100, 2)

    final_value = funds + (holdings * float(df['Close'].iloc[start_idx + simulation_period]))
    return round((final_value / initial_funds - 1) * 100, 2), df_res, final_value, pd.DataFrame(trade_history)




In [59]:

if __name__ == "__main__":
    # 1. 초기 설정
    start_date = '2020-01-01'
    end_date = '2020-12-31'
    initial_funds = 100000
    buy_portion = 7
    ticker = 'TQQQ'




    print("=== 백테스트 시작 ===")
    print(f"종목: {ticker}")
    print(f"기간: {start_date} ~ {end_date}")
    print(f"초기자금: ${initial_funds:,}")
    print(f"분할 횟수: {buy_portion}")

    try:
        # 2. 데이터 준비 - 디버깅을 위한 출력 추가
        start_date_dt = pd.to_datetime(start_date)
        start_date_before_30 = (start_date_dt - pd.Timedelta(days=30)).strftime('%Y-%m-%d')
        end_date_dt = pd.to_datetime(end_date)
        next_day = (end_date_dt + pd.Timedelta(days=1)).strftime('%Y-%m-%d')

        print("\n[디버그] 날짜 정보:")
        print(f"원본 시작일: {start_date}")
        print(f"30일 전 시작일: {start_date_before_30}")
        print(f"원본 종료일: {end_date}")
        print(f"다음 날: {next_day}")

        print("\n데이터 준비 중...")
        df = get_data(ticker, start_date_before_30, next_day)
        print(f"\n[디버깅] 데이터프레임 정보:")
        print(f"인덱스 타입: {type(df.index)}")
        print(f"첫 날짜: {df.index[0]}, 타입: {type(df.index[0])}")
        print(f"마지막 날짜: {df.index[-1]}, 타입: {type(df.index[-1])}")

        # 지표 계산 및 열 이름 단일화
        df = calculate_indicators(df)
        df.columns = df.columns.get_level_values(0)  # MultiIndex를 단일 인덱스로 변환

        print("\n[디버깅] 변환된 열 목록:")
        print(df.columns)

        # 3. 기존 전략 백테스트
        actual_start_data = get_data(ticker, start_date, next_day)
        df_length = len(df) - len(actual_start_data)
        print(f"\n총 {len(df):,}개의 데이터 준비 완료")

        print("\n기존 전략 백테스트 수행 중...")
        baseline_return, baseline_df_res, baseline_final, baseline_trades = infinite_buy_simulation(
            df, pd.DataFrame(), initial_funds, buy_portion, df_length, len(df)-1-df_length)

        print("\n[디버깅] baseline_trades 정보:")
        print(baseline_trades.head())
        print("거래 데이터 수:", len(baseline_trades))


        print("\n기존 전략 결과:")
        print(f"최종 수익률: {baseline_return:.2f}%")
        print(f"최종 자산: ${baseline_final:,.2f}")
        print(f"총 거래 수: {len(baseline_trades)}")
        print(f"성공 거래 비율: {(baseline_trades['수익률(%)'] >= 0).mean():.2%}")

        # 4. ML 모델 준비
        print("\n머신러닝 모델 준비 중...")
        feature_columns = [
            'MACD', 'MACD_Hist', 'MA50이격도', 'MA200이격도',
            '수익률_5', '수익률_60', 'ATR', 'BB_Width',
            '변동성_20', '거래량_증감', '거래량_비율',
            '가격위치_20', '가격위치_60', '캔들크기',
            '연속상승', 'RSI'
        ]

        # 학습 데이터 생성
        df_features = create_training_features(df, baseline_trades)

        # df_features 생성 후 열 점검
        print("\n[디버그] df_features 열 목록:")
        print(df_features.columns)

        # 누락된 열 확인
        missing_columns = [col for col in feature_columns if col not in df_features.columns]
        if missing_columns:
            print(f"\n누락된 열: {missing_columns}")


        # 학습/검증 데이터 분할
        train_size = int(len(df_features) * 0.7)
        df_train = df_features.iloc[:train_size]
        df_validate = df_features.iloc[train_size:]

        X_train = df_train[feature_columns]
        y_train = df_train['is_target_achieved']
        X_validate = df_validate[feature_columns]
        y_validate = df_validate['is_target_achieved']

        # 스케일링
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_validate_scaled = scaler.transform(X_validate)

        # 모델 학습
        model = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)
        model.fit(X_train_scaled, y_train)

        # 5. 모델 성능 분석
        print("\n모델 성능 분석 중...")
        metrics, prob_analysis = analyze_model_performance(
            model, X_validate_scaled, y_validate)

        print("\n[검증 데이터 성능]")
        for metric, value in metrics.items():
            print(f"{metric}: {value:.2%}")

        print("\n[확률 구간별 실제 성공률]")
        print(prob_analysis)

        # 6. 특성 중요도 분석
        importance = analyze_feature_importance(model, feature_columns)
        print("\n[특성 중요도]")
        print(importance)

        # 7. ML 기반 전략 백테스트
        print("\nML 기반 전략 백테스트 수행 중...")
        prediction_threshold = 0.75
        ml_return, ml_df_res, ml_final, ml_trades = ml_infinite_buy_simulation(
            df, pd.DataFrame(), initial_funds, buy_portion, df_length, len(df)-1-df_length,
            model, scaler, prediction_threshold)

        # 8. 최종 결과 비교
        print("\n=== 최종 결과 비교 ===")
        print("\n[기존 전략]")
        print(f"최종 수익률: {baseline_return:.2f}%")
        print(f"최종 자산: ${baseline_final:,.2f}")
        print(f"총 거래 수: {len(baseline_trades)}")
        print(f"성공 거래 비율: {(baseline_trades['수익률(%)'] >= 0).mean():.2%}")

        print("\n[ML 기반 전략]")
        print(f"최종 수익률: {ml_return:.2f}%")
        print(f"최종 자산: ${ml_final:,.2f}")
        print(f"총 거래 수: {len(ml_trades)}")
        print(f"성공 거래 비율: {(ml_trades['수익률(%)'] >= 0).mean():.2%}")

        improvement = ml_return - baseline_return
        print(f"\n수익률 개선: {improvement:.2f}%p")

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        import traceback
        print("\n[디버그] 상세 오류:")
        print(traceback.format_exc())


[*********************100%***********************]  1 of 1 completed

=== 백테스트 시작 ===
종목: TQQQ
기간: 2020-01-01 ~ 2020-12-31
초기자금: $100,000
분할 횟수: 7

[디버그] 날짜 정보:
원본 시작일: 2020-01-01
30일 전 시작일: 2019-12-02
원본 종료일: 2020-12-31
다음 날: 2021-01-01

데이터 준비 중...



[디버깅] 데이터프레임 정보:
인덱스 타입: <class 'pandas.core.indexes.base.Index'>
첫 날짜: 2019-12-02, 타입: <class 'str'>
마지막 날짜: 2020-12-31, 타입: <class 'str'>


[*********************100%***********************]  1 of 1 completed


[디버깅] 변환된 열 목록:
Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'Return', 'MA20',
       'MA50', 'MA50이격도', 'MA200', 'MA200이격도', 'MACD', 'Signal', 'MACD_Hist',
       '수익률_5', '수익률_10', '수익률_20', '수익률_60', 'ATR', 'BB_Middle', 'BB_Std',
       'BB_Upper', 'BB_Lower', 'BB_Width', '변동성_20', '거래량_증감', '거래량_MA20',
       '거래량_비율', 'OBV', '가격위치_20', '가격위치_60', '캔들크기', '일간상승', '연속상승', 'RSI'],
      dtype='object', name='Price')

총 274개의 데이터 준비 완료

기존 전략 백테스트 수행 중...



[디버깅] baseline_trades 정보:
   회차         매수일    매수가  매수수량         매도일    매도가  매도수량  보유기간  수익률(%)
0   2  2020-01-03  22.09   646  2020-01-16  24.61   646    10   11.41
1   3  2020-01-06  22.51   634  2020-01-17  24.98   634    10   10.97
2   4  2020-01-07  22.49   635  2020-01-17  24.98   635     9   11.07
3   1  2020-01-02  22.71   629  2020-01-22  25.13   629    14   10.66
4   5  2020-01-08  23.00   621  2020-01-23  25.36   621    11   10.26
거래 데이터 수: 121

기존 전략 결과:
최종 수익률: 112.25%
최종 자산: $212,248.72
총 거래 수: 121
성공 거래 비율: 87.60%

머신러닝 모델 준비 중...

[디버깅] 생성된 features_list 길이: 121

[디버그] df_features 열 목록:
Index(['회차', 'MACD', 'MACD_Hist', 'MA50이격도', 'MA200이격도', '수익률_5', '수익률_60',
       'ATR', 'BB_Width', '변동성_20', '거래량_증감', '거래량_비율', '가격위치_20', '가격위치_60',
       '캔들크기', '연속상승', 'RSI', '실제수익률', 'is_target_achieved'],
      dtype='object')

모델 성능 분석 중...

[검증 데이터 성능]
정확도: 72.97%
정밀도: 72.97%
재현율: 100.00%
F1: 84.38%

[확률 구간별 실제 성공률]
      확률구간  거래수     실제성공률
0  0.8~0.9    4  1.000000
1  0.9

=== 백테스트 시작 ===
종목: TQQQ
기간: 2011-01-01 ~ 2024-10-31
초기자금: $100,000
분할 횟수: 30

[디버그] 날짜 정보:
원본 시작일: 2011-01-01
30일 전 시작일: 2010-12-02
원본 종료일: 2024-10-31
다음 날: 2024-11-01

데이터 준비 중...


[*********************100%***********************]  1 of 1 completed



[디버그] 데이터프레임 정보:
인덱스 타입: <class 'pandas.core.indexes.base.Index'>
첫 날짜: 2010-12-02, 타입: <class 'str'>
마지막 날짜: 2024-10-31, 타입: <class 'str'>


[*********************100%***********************]  1 of 1 completed
<ipython-input-5-c070036ad75d>:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  open_price = float(df['Open'].iloc[i])
<ipython-input-5-c070036ad75d>:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  high_price = float(df['High'].iloc[i])
<ipython-input-5-c070036ad75d>:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  close_price = float(df['Close'].iloc[i])
<ipython-input-5-c070036ad75d>:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prev_price = float(df['Close'].iloc[i-1]) if i > 0 else close_price



[디버그] 데이터 길이:
전체 데이터: 3502
실제 시작일 데이터: 3481
차이(df_length): 21

총 3,502개의 데이터 준비 완료

기존 전략 백테스트 수행 중...


<ipython-input-5-c070036ad75d>:105: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  final_value = funds + (holdings * float(df['Close'].iloc[start_idx + simulation_period]))



기존 전략 결과:
최종 수익률: 437.93%
최종 자산: $537,928.54
총 거래 수: 3456
성공 거래 비율: 73.84%

머신러닝 모델 준비 중...
Error processing trade 2: datetime.date(2011, 1, 4)
Error processing trade 1: datetime.date(2011, 1, 3)
Error processing trade 3: datetime.date(2011, 1, 5)
Error processing trade 14: datetime.date(2011, 1, 21)
Error processing trade 19: datetime.date(2011, 1, 28)
Error processing trade 4: datetime.date(2011, 1, 6)
Error processing trade 5: datetime.date(2011, 1, 7)
Error processing trade 6: datetime.date(2011, 1, 10)
Error processing trade 7: datetime.date(2011, 1, 11)
Error processing trade 13: datetime.date(2011, 1, 20)
Error processing trade 20: datetime.date(2011, 1, 31)
Error processing trade 15: datetime.date(2011, 1, 24)
Error processing trade 8: datetime.date(2011, 1, 12)
Error processing trade 9: datetime.date(2011, 1, 13)
Error processing trade 12: datetime.date(2011, 1, 19)
Error processing trade 16: datetime.date(2011, 1, 25)
Error processing trade 17: datetime.date(2011, 1, 26)
Err